In [1]:
import os
os.environ["SPS_HOME"] = "/Users/fpetri/packages/fsps" 

In [2]:
import numpy as np
import speculator
from speculator import  Photulator
from speculator import train_photulator_stack
import fsps

In [3]:
# architecture
n_layers = 4
n_units = 128

In [4]:
filter_list = fsps.find_filter('lsst') + fsps.find_filter('suprimecam')[1:2]+fsps.find_filter('suprimecam')[3:]
filter_list

['lsst_u',
 'lsst_g',
 'lsst_r',
 'lsst_i',
 'lsst_z',
 'lsst_y',
 'suprimecam_g',
 'suprimecam_r',
 'suprimecam_i',
 'suprimecam_z']

In [5]:
train_photulator_stack(training_theta, 
                       training_mag[:,select:], 
                       parameters_shift, 
                       parameters_scale, 
                       magnitudes_shift[select:], 
                       magnitudes_scale[select:],
                       filters=filters[select:],
                       n_layers=n_layers,
                       n_units=n_units,
                       validation_split=validation_split,
                       lr=lr,
                       batch_size=batch_size,
                       gradient_accumulation_steps=gradient_accumulation_steps,
                       epochs=epochs,
                       patience=patience,
                       verbose=True, 
                       root_dir=root_dir + 'trained_models/')

NameError: name 'training_theta' is not defined